In [1]:
import pyspark
import pandas as pd
from pyarrow import fs, hdfs
from pyspark.sql.functions import col, split, explode
from pyspark.sql import SparkSession

hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode", port=9000)

/tmp/ipykernel_1047/1868114216.py:7: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs.HadoopFileSystem("hdfs://namenode:9000")


In [2]:

conexao.get_file_info(fs.FileSelector('/', recursive = True))

[<FileInfo for '/Somativa2': type=FileType.Directory>,
 <FileInfo for '/Somativa2/sentiment140.csv': type=FileType.File, size=44326223>]

In [ ]:

conexao.create_dir('SUAPASTA')

In [3]:

caminho_arquivo_origem = '/home/jovyan/work/sentiment140.csv'
caminho_arquivo_destino = '/SUAPASTA/sentiment140.csv'


In [4]:


with conexao.open_output_stream(caminho_arquivo_destino) as stream:
    stream.write(open(caminho_arquivo_origem, 'rb').read())

In [5]:
with conexao.open_output_stream(caminho_arquivo_destino) as stream:
    with open(caminho_arquivo_origem, 'rb') as f:
        stream.write(f.read())

In [6]:

with conexao.open_input_stream(caminho_arquivo_destino) as f:
    print(f.read(50))


b'\xef\xbb\xbftarget;ids;date;flag;user;text\r\n0;1467810369;Mon Apr 06 22:19:45 PDT 2009;NO_QUERY;_TheSpecialOne'


In [7]:
spark = SparkSession.builder.appName('Spark').config('spark.sql.legacy.timeParserPolicy', 'LEGACY').getOrCreate()


In [8]:
df_spark_hdfs = spark.read.csv('hdfs://namenode:9000/Somativa2', header = True, inferSchema = True, sep = ';')

In [10]:
df_words = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('word'))


df_word_counts = df_words.filter(col('word') != '').groupBy('word').count().orderBy(col('count').desc())

df_word_counts.show(10)



+----+------+
|word| count|
+----+------+
|  to|124781|
|   I|117877|
| the| 98203|
|   a| 70968|
|  my| 65431|
|   i| 62713|
| and| 56596|
|  is| 47798|
|  in| 43863|
| for| 38425|
+----+------+
only showing top 10 rows



In [11]:
df_words = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('word'))


df_mentions = df_words.filter(col('word').startswith('@')).groupBy('word').count().orderBy(col('count').desc())


df_mentions.show(10)

+----------------+-----+
|            word|count|
+----------------+-----+
|               @| 2217|
|     @mileycyrus|  673|
|       @tommcfly|  658|
|       @ddlovato|  407|
|@JonathanRKnight|  216|
|  @taylorswift13|  193|
|   @mitchelmusso|  189|
|    @DavidArchie|  175|
|   @jordanknight|  156|
|    @selenagomez|  142|
+----------------+-----+
only showing top 10 rows



In [12]:
df_words = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('word'))


df_hashtags = df_words.filter(col('word').startswith('#')).groupBy('word').count().orderBy(col('count').desc())

df_hashtags.show(10)

+---------------+-----+
|           word|count|
+---------------+-----+
|            #fb|  371|
|       #asot400|  111|
|          #fail|  110|
|              #|   85|
|           #bgt|   71|
|         #NOME?|   55|
|#whatsmysection|   52|
|    #myweakness|   50|
|             #1|   49|
|  #marsiscoming|   48|
+---------------+-----+
only showing top 10 rows

